# Import Libraries

In [207]:
import numpy as np 
import pandas as pd
import warnings 
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

## Upload Unprocessed CSV file

In [157]:
df = pd.read_csv('Unprocessed Job Data.csv', sep=";", encoding='latin-1', escapechar='\\')

In [159]:
df.head()

,Job_title,Company_name,Location,Salary_fork,Rating,Company_Overview,Job_description,"Avg_base_salary,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
0,Data Engineer Flink,Egrove Systems Corporation,"Dallas, TX",Employer Provided Salary:$51.83 Per Hour,4.4,"Company Overview,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...","Java, Bigdata, Azure & Flink,,,,,,,,,,,,,,,,,,...","Average Base Salary Estimate,,,,,,,,,,,,,,,,,,..."
1,Data Engineer,Overstock.com Inc.,Remote,Employer Provided Salary:$87K - $108K,3.1,"Company Overview,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...","We Are Overstock:,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...","Average Base Salary Estimate,,,,,,,,,,,,,,,,,,..."
2,Data Engineer I,Nucor Business Technology,"Charlotte, NC",$67K - $86K (Glassdoor est.),3.9,"Company Overview,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...","Job Details,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...","Average Base Salary Estimate,,,,,,,,,,,,,,,,,,..."
3,Data Engineer 1,Kagr Llc,"Foxborough, MA",$70K - $98K (Glassdoor est.),3.4,"Company Overview,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...","SUMMARY:,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...","Average Base Salary Estimate,,,,,,,,,,,,,,,,,,..."
4,BI Data & Visualization Engineer 1,G&J Pepsi-Cola Bottlers,"Columbus, OH",$65K - $96K (Glassdoor est.),3.6,"Company Overview,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...","Key Responsibilities,,,,,,,,,,,,,,,,,,,,,,,,,,...","Average Base Salary Estimate,,,,,,,,,,,,,,,,,,..."


In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3766 entries, 0 to 3765
Data columns (total 8 columns):
 #   Column                                                                                                                                                                                                                          Non-Null Count  Dtype 
---  ------                                                                                                                                                                                                                          --------------  ----- 
 0   Job_title                                                                                                                                                                                                                       3766 non-null   object
 1   Company_name                                                                                                                         

### 1. Create a function to generate ids and make a new column named JOB_ID using it.


In [163]:
def generate_job_id(index):
    """Generates a unique job ID in the format JOB0001, JOB0002, etc."""
    return f"JOB{str(index + 1).zfill(4)}"

# Assuming your DataFrame is named 'df'
df['JOB_ID'] = df.index.map(generate_job_id)

In [197]:
df['JOB_ID'].head()

0    JOB0001
1    JOB0002
2    JOB0003
3    JOB0004
4    JOB0005
Name: JOB_ID, dtype: object

In [165]:
missing_values = df.isnull().sum()
print(missing_values)

Job_title                                                                                                                                                                                                                         0
Company_name                                                                                                                                                                                                                      0
Location                                                                                                                                                                                                                          4
Salary_fork                                                                                                                                                                                                                       0
Rating                                                                                  

Covert Job_description column data into String 

In [169]:
df['Job_description'] = df['Job_description'].astype(str)

### 2. Create a function to clean the messy text and use this function to make a new column named Clean_Job_Descripton.

In [191]:
# Clean the Job_description column
df['Clean_Job_description'] = df['Job_description'].str.replace('\r\n', '', regex=False).str.strip().str.replace('"', '')
df['Clean_Job_description'] = df['Job_description'].apply(lambda x: re.sub(r',+', '', str(x)))

In [195]:
df['Clean_Job_description'].head()

0    Java   Bigdata   Azure & Flink  Flink Sr. Deve...
1    We Are Overstock:  At Overstock.com   we belie...
2    Job Details  Division: Nucor Business Technolo...
3    SUMMARY:  The Data Engineer 1 position will be...
4    Key Responsibilities  Leads development of rep...
Name: Job_description, dtype: object

### 3. Create a function to filter out skills from the Clean_Job_Descripton column and make a separate column named skills that stores skills in a list format.


In [201]:
from nltk.corpus import stopwords

def extract_skills(text):
    """Extracts skills from a text string."""
    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Filter for potential skills based on criteria (e.g., length, part of speech)
    skills = [word for word in filtered_tokens if len(word) > 3]

    return skills

df['Skills'] = df['Job_description'].apply(extract_skills)

In [203]:
df['Skills'].head()

0    [Java, Bigdata, Azure, Flink, Flink, Developer...
1    [Overstock, Overstock.com, believe, everyone, ...
2    [Details, Division, Nucor, Business, Technolog...
3    [SUMMARY, Data, Engineer, position, part, dyna...
4    [Responsibilities, Leads, development, reports...
Name: Skills, dtype: object